In [1]:
import xmltodict
import subprocess

In [2]:
with open("./STM32F7_svd/STM32F7_svd_V1.4/STM32F7x6.svd",'r') as xf:
    desc = xmltodict.parse(xf.read())

In [9]:
cpp_field_template = "static constexpr bitfield_t<{reg_type},{start},{stop}> {name} = {{}};\n"
cpp_field_1bit_template = "static constexpr bitfield_t<{reg_type},{start}> {name} ={{}};\n"

cpp_reg_template = "static constexpr struct {reg_type}:reg_t<{str_type}, base_address+{offset}>\n{{ {fields}using reg_t<{str_type}, base_address+{offset}>::operator=;\n}}{name}={{}};\n"
cpp_reg_no_field_template = "static constexpr reg_t<{str_type}, base_address+{offset}> {name}={{}};\n"

cpp_periph_template = "template<uint32_t base_address, int peripheral_index>\n struct {name}_t\n\
{{\n using tag = ucpp::tags::{tag};\n static constexpr int index = peripheral_index;\n static constexpr int address = base_address; \n {registers} }};\n"

c_reg_template = "volatile {reg_type} {name};\n"
c_periph_template = "struct {name}_c_t\n{{\n {registers} }};\n"



In [10]:
def prnt(item, indent):
    if hasattr(item,'prnt'):
        return item.prnt(indent)
    elif type(item) is dict:
        return"".join([
            (" "*indent) +name+": "+prnt(subitem, indent+2)+"\n" for name,subitem in item.items()
        ])
    else:
        return str(item)
        


class SVDElement:
    def __init__(self, dic, parse_table):
        for name,value in dic.items():
            if type(name) is str:
                self.__dict__[name.replace('@','')]=parse_table.get(name, lambda s,p : str(s) )(value, parse_table)
    
    def prnt(self, indent=0):
        return"".join([(" "*indent) +name+": "+prnt(item, indent+2)+'\n' for name,item in self.__dict__.items()])
        
    def __repr__(self):
        return self.prnt()

class Field(SVDElement):
    def __init__(self, field, parse_table):
        super(Field, self).__init__(field, parse_table)
        
    def to_cpp(self, reg_type):
        start = self.bitOffset
        stop = start + self.bitWidth - 1
        if self.bitWidth == 32:
            print(start,stop, self.name)
        if start != stop:
            return cpp_field_template.format(start = start, stop=stop,reg_type=reg_type, name=self.name)
        return cpp_field_1bit_template.format(start = start, reg_type=reg_type, name=self.name)

class Register(SVDElement):
    def __init__(self, reg, parse_table):
        super(Register, self).__init__(reg, parse_table)
        
    def __eq__(self, other):
        return self.name == other.name
    
    def to_cpp(self):
        name = self.name
        reg_type = self.name+"_t"
        offset = self.addressOffset
        if self.size == 32:
            str_type = 'uint32_t'
        elif self.size == 16:
            str_type = 'uint16_t'
        if len(self.fields)==1 and next(iter(self.fields.values())).bitWidth == self.size:
            return cpp_reg_no_field_template.format(str_type=str_type,offset=offset, name=name)
        fields = "".join([bitfield.to_cpp(reg_type) for bitfield in self.fields.values()])
        return cpp_reg_template.format(reg_type=reg_type, str_type=str_type,offset=offset, name=name, fields=fields)
    
    def to_c(self):
        name = self.name
        offset = self.addressOffset
        if self.size == 32:
            str_type = 'uint32_t'
        elif self.size == 16:
            str_type = 'uint16_t'
        return c_reg_template.format(reg_type=str_type,name=name)
        
class Peripheral(SVDElement):
    def __init__(self,periph, parse_table):
        super(Peripheral, self).__init__(periph, parse_table)
    def __eq__(self, other):
        return self.registers.keys() == other.registers.keys()
    
    def to_cpp(self, tag, name=None):
        if not name:
            name=self.name
        registers="".join([register.to_cpp() for register in self.registers.values()])
        return cpp_periph_template.format(name=name, registers=registers, tag=tag)
            
    def to_c(self, name=None):
        if not name:
            name=self.name
        offset = 0
        registers = ""
        for register in self.registers.values():
            while offset < register.addressOffset:
                registers += c_reg_template.format(reg_type="uint32_t",name="__dummy"+str(offset))
                offset += 4
            registers += register.to_c()
            offset += int(register.size/8)
        return c_periph_template.format(name = name, registers=registers)
        
        
class Device(SVDElement):
    def __init__(self,dev , parse_table):
        super(Device, self).__init__(dev, parse_table)
        
_ctors_parse_table={
    'field': Field,
    'register':Register,
    'peripheral':Peripheral,
}

def listify(item_or_list):
    if type(item_or_list) is list:
        return item_or_list
    else:
        return [item_or_list]

def ctor(key, dic, parse_table):
    return _ctors_parse_table[key](dic[key], parse_table)

_parse_table={
    'addressOffset':lambda s,p : int(s,16),
    'bitOffset':lambda s,p : int(s),
    'bitWidth':lambda s,p : int(s),
    'size': lambda s,p  : int(s,16),
    'resetValue': lambda s,p : int(s,16),
    'fields': lambda item,parse_table : { field['name']:Field(field,parse_table) for field in listify(item["field"])},
    'registers': lambda item,parse_table : { register['name']:Register(register,parse_table) for register in listify(item["register"])},
    'peripherals': lambda item,parse_table : { peripheral['name']:Peripheral(peripheral,parse_table) for peripheral in listify(item["peripheral"])},
}

d = Device(desc['device'],_parse_table)
for p in d.peripherals.values():
    if hasattr(p,'derivedFrom'):
        p.__dict__['registers']=d.peripherals[p.derivedFrom].registers

In [11]:
def all_same(device, periph_name):
    periphs = [p for n,p in device.peripherals.items() if periph_name in n]
    return all([[periph]*len(periphs)==periphs for periph in periphs])

all_same(d,'GPIO')
all_same(d,'SPI')

False

In [12]:
d.peripherals.keys()

dict_keys(['RNG', 'HASH', 'CRYP', 'DCMI', 'FMC', 'DBG', 'DMA2', 'DMA1', 'RCC', 'GPIOD', 'GPIOC', 'GPIOK', 'GPIOJ', 'GPIOI', 'GPIOH', 'GPIOG', 'GPIOF', 'GPIOE', 'GPIOB', 'GPIOA', 'SYSCFG', 'SPI1', 'SPI3', 'SPI4', 'SPI5', 'SPI6', 'SPI2', 'ADC1', 'ADC2', 'ADC3', 'DAC', 'PWR', 'IWDG', 'WWDG', 'C_ADC', 'TIM1', 'TIM8', 'TIM2', 'TIM3', 'TIM4', 'TIM5', 'TIM9', 'TIM12', 'TIM10', 'TIM11', 'TIM13', 'TIM14', 'TIM6', 'TIM7', 'Ethernet_MAC', 'Ethernet_MMC', 'Ethernet_PTP', 'Ethernet_DMA', 'CRC', 'CAN1', 'CAN2', 'FLASH', 'EXTI', 'LTDC', 'SAI1', 'SAI2', 'DMA2D', 'QUADSPI', 'CEC', 'SPDIF_RX', 'SDMMC1', 'LPTIM1', 'I2C1', 'I2C2', 'I2C3', 'I2C4', 'RTC', 'USART6', 'USART1', 'USART3', 'USART2', 'UART5', 'UART4', 'UART8', 'UART7', 'OTG_FS_GLOBAL', 'OTG_FS_HOST', 'OTG_FS_DEVICE', 'OTG_FS_PWRCLK', 'OTG_HS_GLOBAL', 'OTG_HS_HOST', 'OTG_HS_DEVICE', 'OTG_HS_PWRCLK', 'NVIC', 'MPU', 'STK', 'NVIC_STIR', 'FPU_CPACR', 'SCB_ACTRL', 'FPU', 'SCB', 'PF', 'AC'])

In [13]:
#clang-format -i -style=file  gpio-regs.hpp 
def make_header(peripheral, tag, name=None):
    if not name:
        name = peripheral.name
    cpp = peripheral.to_cpp(tag, name)
    c   = peripheral.to_c(name)
    fname = f'{name.lower()}-regs.hpp'
    with open(fname, 'w') as f:
        f.write(f"""
    #pragma once
    #include <stdint.h>
    #include "../register.hpp"
    #include "../peripherals_tags.hpp"


    using namespace ucpp::registers;
    namespace ucpp::stm32::{name.lower()} {{
    {cpp}

    {c}
    }}
        """)
    
    subprocess.run(["clang-format", "-i", '-style=file', fname])

In [14]:
make_header(d.peripherals["RCC"], "no_tag")
make_header(d.peripherals["GPIOA"],"gpio_tag","gpio")
make_header(d.peripherals["SDMMC1"],"sdmmc_tag", "sdmmc")

In [ ]:
rcc = d.peripherals["RCC"].to_cpp()
rcc_c = d.peripherals["RCC"].to_c()
with open('rcc-regs.hpp', 'w') as f:
    f.write(f"""
#pragma once
#include <stdint.h>
#include "../register.hpp"
#include "../peripherals_tags.hpp"

using namespace ucpp::registers;
namespace ucpp::stm32::rcc {{
{rcc}

{rcc_c }
}}
    """)

In [ ]:
gpio = d.peripherals["GPIOA"].to_cpp("GPIO")
gpio_c = d.peripherals["GPIOA"].to_c()
with open('gpio-regs.hpp', 'w') as f:
    f.write(f"""
#pragma once
#include <stdint.h>
#include "../register.hpp"


using namespace ucpp::registers;
namespace ucpp::stm32::gpio {{
{gpio}

{gpio_c }
}}
    """)

In [ ]:
d.peripherals.keys()

In [ ]:
sd = d.peripherals["SDMMC1"]

In [ ]:
sd.registers["ARG"].fields["CMDARG"].bitOffset